In [7]:
import pandas as pd

df = pd.read_csv('../data/creditcard.csv/cleaned_creditcard.csv')
df.columns #notice how this is the new clean

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class', 'Amount_scaled'],
      dtype='object')

Apply SMOTE

#Split the data into training and testing sets

In [20]:
#Input features = All columns except Class
#Target Variable = Class

X = df.drop(columns='Class')
y = df['Class']


In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, stratify=y, random_state=42)

print(X_train.shape, X_test.shape)
print(y_train.value_counts(), "\n")
print(y_test.value_counts())

(226980, 31) (56746, 31)
Class
0    226602
1       378
Name: count, dtype: int64 

Class
0    56651
1       95
Name: count, dtype: int64


In [ ]:
#Now we want to apply SMOTE X_train and y_train
#This should add more training data to our x nd y train_resampled so that we have more data to train from since the data is imbalanced
#It generates new synthetic samples for the minority class only
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state = 42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print(y_train.value_counts())
print(y_train_resampled.value_counts())


Class
0    226602
1       378
Name: count, dtype: int64
Class
0    226602
1    226602
Name: count, dtype: int64
